## LM-branch predictor数据处理

把raw的trace文件转换成句子文件

In [9]:
def trace2sentence(filename, window_length):
    trace_file = open("D:/tensorflow/Branch_Prediction/RNN_Branch_prediction/data/trace/trace_file.txt", 'w')
    with open(filename) as f:
        lines = f.readlines()
        for num in range(len(lines)-window_length+1):
            for items in lines[num:num+window_length]:
                words = items.strip().split(',')
                word = words[0] + words[1]
                trace_file.write(word + ' ')
            trace_file.write('\n')
    trace_file.close()
    
trace2sentence('SHORT_SERVER-103.bt9.trace', 5)

分割训练集，验证集，测试集

In [10]:
def split_dataset(filename, percent):
    train_file = open("D:/tensorflow/Branch_Prediction/RNN_Branch_prediction/data/trace/train.txt",'w')
    dev_file = open("D:/tensorflow/Branch_Prediction/RNN_Branch_prediction/data/trace/dev.txt", 'w')
    test_file = open("D:/tensorflow/Branch_Prediction/RNN_Branch_prediction/data/trace/test.txt", 'w')
    with open(filename) as f:
        lines = f.readlines()
        num_lines = int(percent * len(lines))
        train_num = int(0.8 * num_lines)
        dev_num = int(0.1 * num_lines)
        test_num = num_lines -train_num - dev_num
        for line in lines[0:train_num+1]:
            train_file.write(line)
        for line in lines[train_num+1: train_num + dev_num + 1]:
            dev_file.write(line)
        for line in lines[train_num + dev_num + 1: num_lines + 1]:
            test_file.write(line)
    train_file.close()
    dev_file.close()
    test_file.close()
    
split_dataset("trace_file.txt", 0.5)

## Text classification数据处理

把raw的trace文件转换成句子文件，并添加标签

In [28]:
def trace2sentenceLabel(filename, window_length):
    trace_file = open("E:/traces/SHORT_SERVER-101/20/trace_file.txt", 'w')
    with open(filename) as f:
        lines = f.readlines()
        for num in range(len(lines)-window_length+1):
            for items in lines[num:num+(window_length-1)]:
                words = items.strip().split(',')
                word = words[0] + words[1]
                trace_file.write(word + ' ')
            words = lines[num+window_length-1].strip().split(',')
            trace_file.write(words[0] + '\t' + words[1])
            trace_file.write('\n')
    trace_file.close()
    
trace2sentenceLabel('SHORT_SERVER-101.bt9.trace', 20)

In [29]:
def split_dataset2(filename, percent):
    train_file = open("E:/traces/SHORT_SERVER-101/20/trace_train.txt",'w')
    dev_file = open("E:/traces/SHORT_SERVER-101/20/trace_val.txt", 'w')
    test_file = open("E:/traces/SHORT_SERVER-101/20/trace_test.txt", 'w')
    with open(filename) as f:
        lines = f.readlines()
        num_lines = int(percent * len(lines))
        train_num = int(0.8 * num_lines)
        dev_num = int(0.1 * num_lines)
        test_num = num_lines -train_num - dev_num
        for line in lines[0:train_num+1]:
            train_file.write(line)
        for line in lines[train_num+1: train_num + dev_num + 1]:
            dev_file.write(line)
        for line in lines[train_num + dev_num + 1: num_lines + 1]:
            test_file.write(line)
    train_file.close()
    dev_file.close()
    test_file.close()
    
split_dataset2("E:/traces/SHORT_SERVER-101/20/trace_file.txt", 1)

In [58]:

import sys
from collections import Counter

import numpy as np
import tensorflow.contrib.keras as kr

def read_file(filename):
    """读取文件数据, contents = [[],[]...]"""
    contents, labels = [], []
    with open(filename,'r') as f:
        for line in f:
            content, label = line.strip().split('\t')
            contents.append(content.strip().split())
            labels.append(label)
    return contents, labels

def build_vocab(train_dir, vocab_dir, vocab_size=5000):
    """根据训练集构建词汇表，存储"""
    data_train, _ = read_file(train_dir)
    data = []
    for item in data_train:
        #data = data+item   
        data.extend(item)
    counter = Counter(data)
    count_pairs = counter.most_common(vocab_size - 1)
    words, _ = list(zip(*count_pairs))
    open(vocab_dir, 'w').write('\n'.join(words) + '\n')
    
def read_vocab(vocab_dir):
    """读取词汇表"""
    with open(vocab_dir, 'r') as fp:
        words = [_.strip() for _ in fp.readlines()]
    word_to_id = dict(zip(words, range(len(words))))
    return words, word_to_id

def read_category():
    """读取分类目录，固定"""
    categories = ['0', '1']
    categories = [x for x in categories]
    cat_to_id = dict(zip(categories, range(len(categories))))
    return categories, cat_to_id


def to_words(content, words):
    """将id表示的内容转换为文字"""
    return ''.join(words[x] for x in content)


def process_file(filename, word_to_id, cat_to_id,max_length = 5):
    """将文件转换为id表示, data_id = [[],[]...]"""
    contents, labels = read_file(filename)
    data_id, label_id = [], []
    for i in range(len(contents)):
        data_id.append([word_to_id[x] for x in contents[i] if x in word_to_id])
        label_id.append(cat_to_id[labels[i]])
    # 使用keras提供的pad_sequences来将文本pad为固定长度
    x_pad = kr.preprocessing.sequence.pad_sequences(data_id, max_length)
    y_pad = kr.utils.to_categorical(label_id, num_classes=len(cat_to_id))  # 将标签转换为one-hot表示
    #return data_id, label_id
    data_id = np.array(data_id)
    label_id = np.array(label_id)
    print(data_id)
    print(label_id)
    return x_pad, y_pad

_, cat_to_id = read_category()
_, word_to_id = read_vocab('vocab.txt')
a, b = process_file('trace_file.txt', word_to_id, cat_to_id)
print(a)
print(b)

[[ 0  3  2  1  5]
 [ 3  2  1  0  7]
 [ 2  1  0  4  8]
 [ 1  0  4  2  9]
 [ 0  4  2  1  5]
 [ 4  2  1  0  7]
 [ 2  1  0  3  8]
 [ 1  0  3  6  9]
 [ 0  3  6  1  5]
 [ 3  6  1  0  7]
 [ 6  1  0  3  8]
 [ 1  0  3  2  9]
 [ 0  3  2  1  5]
 [ 3  2  1  0  7]
 [ 2  1  0  3  8]
 [ 1  0  3  2  9]
 [ 0  3  2  1  5]
 [ 3  2  1  0  7]
 [ 2  1  0  3  8]
 [ 1  0  3  2  9]
 [ 0  3  2  1  5]
 [ 3  2  1  0  7]
 [ 2  1  0  4  8]
 [ 1  0  4  2  9]
 [ 0  4  2  1  5]
 [ 4  2  1  0  7]
 [ 2  1  0  4  8]
 [ 1  0  4  6  9]
 [ 0  4  6 10  5]
 [ 4  6 10  0  7]
 [ 6 10  0  3  8]
 [10  0  3  6  9]
 [ 0  3  6  1  5]
 [ 3  6  1  0  7]
 [ 6  1  0  3  8]
 [ 1  0  3  2  9]
 [ 0  3  2  1  5]
 [ 3  2  1  0  7]
 [ 2  1  0  4  8]
 [ 1  0  4  2  9]
 [ 0  4  2  1  5]
 [ 4  2  1  0  7]
 [ 2  1  0  3  8]
 [ 1  0  3  2  9]
 [ 0  3  2  1  5]
 [ 3  2  1  0  7]]
[1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 0 0 1 1 0 1 1 1 1 1 1
 0 1 1 1 1 1 1 1 0]
[[ 0  3  2  1  5]
 [ 3  2  1  0  7]
 [ 2  1  0  4  8]
 [ 1  0  4  2  9]
 [ 0

In [50]:
a = [[1]]
a = np.array(a)
a.shape

(1, 1)